In [1]:
import sys
sys.executable

'c:\\Users\\carlos.anton\\OneDrive - Grupo VASS\\Carlos\\Desarrollo_personal\\CVInsights-Project\\venv\\Scripts\\python.exe'

In [2]:
import spacy
from spacy import displacy
nlp = spacy.load("en_core_web_sm")

text = "IN THE MATTER OF a proposed contract between the Department of Citywide Administrative Services of the City of New York and Tesla, Inc., located at 3500 Deer Creek Rd., Palo Alto, CA 94304, for procuring Tesla Model 3 All-Electric Sedans. The contract is in the amount of $12,360,000.00. The term of the contract shall be five years from date of Notice of Award. The proposed contractor has been selected by Sole Source Procurement Method, pursuant to Section 3-05 of the Procurement Policy Board Rules. If the plan does go through, the $12.36 million could effectively purchase about 274 units of the base Model 3 Rear-Wheel-Drive, which cost $44,990 under Tesla's current pricing structure."

doc = nlp(text)
displacy.render(doc, style="ent")

c:\Users\carlos.anton\OneDrive - Grupo VASS\Carlos\Desarrollo_personal\CVInsights-Project\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\carlos.anton\OneDrive - Grupo VASS\Carlos\Desarrollo_personal\CVInsights-Project\venv\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\carlos.anton\OneDrive - Grupo VASS\Carlos\Desarrollo_personal\CVInsights-Project\venv\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [55]:
import json
import logging

def convert_dataturks_to_spacy(dataturks_JSON_FilePath):
    try:
        TRAIN_DATA = []
        with open(dataturks_JSON_FilePath, 'r', encoding="utf8") as f:
            data = json.load(f)
            for item in data:
                text = item['content']
                entities = []
                for annotation in item['annotation']:
                    point = annotation['points'][0]
                    labels = annotation['label']
                    if not isinstance(labels, list):
                        labels = [labels]
                    for label in labels:
                        entities.append((point['start'], point['end'] + 1, label))
                TRAIN_DATA.append((text, {"entities": entities}))
        return TRAIN_DATA
    except Exception as e:
        logging.exception("Unable to process " + dataturks_JSON_FilePath + "\n" + "error = " + str(e))
        return None

In [3]:
# Consultamos las capacidades del modelo
print(nlp.pipe_names)

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [5]:
# Consultar información sobre las entidades nombradas (NER)
print(nlp.get_pipe("ner").labels)

('CARDINAL', 'DATE', 'EVENT', 'FAC', 'GPE', 'LANGUAGE', 'LAW', 'LOC', 'MONEY', 'NORP', 'ORDINAL', 'ORG', 'PERCENT', 'PERSON', 'PRODUCT', 'QUANTITY', 'TIME', 'WORK_OF_ART')


In [56]:
import json
import logging

TRAIN_DATA = convert_dataturks_to_spacy("traindata_array.json")

In [68]:
TRAIN_DATA[0:1]

[('Vijayalakshmi Govindarajan\nSAP as a Consultant - SAP Basis\n\nChennai, Tamil Nadu - Email me on Indeed: indeed.com/r/Vijayalakshmi-Govindarajan/\nd71bfb70a66b0046\n\nWilling to relocate: Anywhere\n\nWORK EXPERIENCE\n\nSAP as a Consultant\n\nSAP Basis -\n\nMay 2012 to Present\n\nInvolved in 3 support Projects and 1 implementation Project.\n• System Build, System Refresh, system upgrade and system maintenance activities.\n• Self-motivated and goal-oriented, with a strong ability to organize and prioritize tasks\nefficiently.\n• Excellent team player with good communication, analytical, problem solving skills.\n• Certification Done: Oracle certified Java Programmer\n• Finished Diploma in Java course from NIIT\n\nEDUCATION\n\nMCA in Computer Applications\n\nThiagarajar School of Management -  Madurai, Tamil Nadu\n\nBSc\n\nSri Sathya Sai Institute of Higher Learning -  Anantapur, Andhra Pradesh\n\nHSC\n\nTVS Lakshmi Matriculation Higher Secondary School -  Madurai, Tamil Nadu\n\nSSLC\n\

In [58]:
# def built_spacy_ner(text, target, type):
#     start = str.find(text, target)
#     end = start + len(target)

#     return (text, {"entities": [(start, end, type)]})

In [59]:
# TRAIN_DATA = []
# TRAIN_DATA.append(
#   built_spacy_ner("I work for Autodesk.", "Autodesk", "ORG")
#   )

In [60]:
# TRAIN_DATA

In [61]:
# TRAIN_DATA.extend([
#     built_spacy_ner("The proposed contract is for the procurement of Tesla Model S Electric Sedans.", "Tesla Model S Electric Sedans", "PRODUCT"),
#     built_spacy_ner("The contract is worth $8,500,000.00.", "$8,500,000.00", "MONEY"),
#     built_spacy_ner("The agreement will last for three years.", "three years", "DURATION"),
#     built_spacy_ner("The selection process follows the guidelines of Section 4-02 of the Procurement Policy Board Rules.", "Section 4-02", "LAW"),
#     built_spacy_ner("If the project proceeds, it could acquire 150 units of the base Model 5 Rear-Wheel-Drive.", "150 units of the base Model 5 Rear-Wheel-Drive", "PRODUCT"),
#     built_spacy_ner("The base Model 5 Rear-Wheel-Drive has a price of $39,990.", "$39,990", "MONEY")
# ])


In [62]:
# TRAIN_DATA

In [63]:
# adding a named entity label
ner = nlp.get_pipe('ner')

# Iterate through training data and add new entitle labels.
for _, annotations in TRAIN_DATA:
  for ent in annotations.get("entities"):
    ner.add_label(ent[2])

In [64]:
# creating an optimizer and selecting a list of pipes NOT to train
optimizer = nlp.create_optimizer()
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

In [65]:
other_pipes

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer']

In [66]:
import random
from spacy.training.example import Example


with nlp.disable_pipes(*other_pipes):
    for itn in range(10):
        random.shuffle(TRAIN_DATA)
        losses = {}

        # batch the examples and iterate over them
        for batch in spacy.util.minibatch(TRAIN_DATA, size=2):
            for text, annotations in batch:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                nlp.update([example], drop=0.35, sgd=optimizer, losses=losses)

print("Final loss: ", losses)

c:\Users\carlos.anton\OneDrive - Grupo VASS\Carlos\Desarrollo_personal\NER_TDS\NER_TDS\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Vijayalakshmi Govindarajan
SAP as a Consultant - S..." with entities "[(6310, 6336, 'Name'), (5746, 5755, 'Companies wor...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\carlos.anton\OneDrive - Grupo VASS\Carlos\Desarrollo_personal\NER_TDS\NER_TDS\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Krishna Prasad
Patna City, Bihar - Email me on Ind..." with entities "[(288, 293, 'Location'), (267, 285, 'College Name'...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warni

ValueError: [E103] Trying to set conflicting doc.ents: '(983, 992, 'Companies worked at')' and '(970, 1002, 'Skills')'. A token can only be part of one entity, so make sure the entities you're setting don't overlap. To work with overlapping entities, consider using doc.spans instead.

In [19]:
text2 = "IN THE MATTER OF a proposed contract between the Department of Citywide Administrative Services of the City of New York and Tesla, Inc., located at 3500 Deer Creek Rd., Palo Alto, CA 94304, for procuring Tesla Model 3 All-Electric Sedans. The contract is in the amount of $12,360,000.00. The term of the contract shall be five years from date of Notice of Award. The proposed contractor has been selected by Sole Source Procurement Method, pursuant to Section 3-05 of the Procurement Policy Board Rules. If the plan does go through, the $12.36 million could effectively purchase about 274 units of the base Model 3 Rear-Wheel-Drive, which cost $44,990 under Tesla's current pricing structure."

doc = nlp(text2)
displacy.render(doc, style="ent", jupyter=True)